In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-20 12:33:05.880283: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 12:33:06.113040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [4]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
init(model)

In [7]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack
from adapters import LoRAConfig

adapter_name= "lora-mlm-target"

adapter_config = LoRAConfig(r=8, alpha=16)
model.add_adapter(adapter_name, config=adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 739584 || all params: 67132986 || trainable%: 1.1016700493554688


In [8]:
model.active_adapters

Stack[lora-mlm-target]

In [9]:
model.adapter_summary(as_dict=True)

[{'name': 'lora-mlm-target',
  'architecture': 'lora',
  'active': True,
  '#param': 147456,
  'train': True,
  '%param': 0.22219650503413957},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [10]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


distilbert.transformer.layer.0.attention.q_lin.loras.lora-mlm-target.lora_A
distilbert.transformer.layer.0.attention.q_lin.loras.lora-mlm-target.lora_B
distilbert.transformer.layer.0.attention.v_lin.loras.lora-mlm-target.lora_A
distilbert.transformer.layer.0.attention.v_lin.loras.lora-mlm-target.lora_B
distilbert.transformer.layer.1.attention.q_lin.loras.lora-mlm-target.lora_A
distilbert.transformer.layer.1.attention.q_lin.loras.lora-mlm-target.lora_B
distilbert.transformer.layer.1.attention.v_lin.loras.lora-mlm-target.lora_A
distilbert.transformer.layer.1.attention.v_lin.loras.lora-mlm-target.lora_B
distilbert.transformer.layer.2.attention.q_lin.loras.lora-mlm-target.lora_A
distilbert.transformer.layer.2.attention.q_lin.loras.lora-mlm-target.lora_B
distilbert.transformer.layer.2.attention.v_lin.loras.lora-mlm-target.lora_A
distilbert.transformer.layer.2.attention.v_lin.loras.lora-mlm-target.lora_B
distilbert.transformer.layer.3.attention.q_lin.loras.lora-mlm-target.lora_A
distilbert.t

In [11]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(unsupervised_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 52350
})

In [12]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [13]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [14]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 18436
})

In [15]:
print(tokenizer.decode(lm_datasets[1]["input_ids"]))
print(tokenizer.decode(lm_datasets[1]['labels']))
print(len(lm_datasets[-1]['labels']))


1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the minoans used images of birds and fish to represent how they thought they would be reincarnated. [SEP] [CLS] shinjuku has tokyo's largest discount camera stores, although these rarely
1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the

In [16]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] foreigners from the entire ottoman empire flooded into galata and pera, attracted by the wealth and sophistication of [MASK] capital. [SEP] no foreigners came from other lands to [MASK]ta. [SEP] [CLS] muslim rule was largely tolerant, and [MASK] peacefully for nearly four centuries with a joint christian [MASK] [MASK] protectorate of holy places. [SEP] christians and muslims were fierce enemies, especially when [MASK] came to holy places. [SEP] [CLS] on boulevard des cap [MASK]ines, retrace [MASK] footsteps of renoir, manet, and 10tharro as they took their paintings [MASK] nadar's house, [MASK]posted at number 35 [MASK] for [MASK] historic'

'>>> restrictive exhibition of impressionism. [SEP] piss [MASK]ro never walked [MASK] the boulevard des capucines. [SEP] [CLS] a [MASK] was once built over this spot, but today the minaret of a small mosque is the [MASK] landmark [MASK] [MASK] the [MASK]. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains 

In [17]:
reload(fn)
samples = [lm_datasets[i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [MASK] [MASK] the entire [MASK] empire [MASK] into [MASK] [MASK] and pera, attracted by the wealth and sophistication of the capital. [SEP] no foreigners came from [MASK] lands to galata. [SEP] [CLS] [MASK] rule was largely tolerant, and continued peacefully [MASK] nearly four [MASK] with a joint christian - muslim protectorate of holy places. [SEP] christians and [MASK] were fierce [MASK], especially [MASK] it came to holy places [MASK] [SEP] [CLS] on [MASK] des capucines, retrace [MASK] footsteps of renoir [MASK] manet, and pissarro as they took their [MASK] to nadar [MASK] s house, signposted at number 35, for the historic'

'>>> 1874 [MASK] of impressionism. [SEP] [MASK] [MASK] [MASK] never walked on the boulevard [MASK] capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of [MASK] small mosque is [MASK] best landmark for finding [MASK] pool. [SEP] just look [MASK] the old [MASK] to find the pool. [SEP] [CLS] room 13 contains a [MASK] of

In [18]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 16592
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1844
    })
})

In [19]:

reload(fn)
trainer = fn.train_mlm_model(model,"lora-mlm-target",data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [20]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 21.30


In [21]:
trainer.train()

  0%|          | 0/5190 [00:00<?, ?it/s]

{'loss': 2.5994, 'learning_rate': 9.961620469083156e-05, 'epoch': 1.0}


  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 2.2025861740112305, 'eval_runtime': 1.9588, 'eval_samples_per_second': 941.389, 'eval_steps_per_second': 29.61, 'epoch': 1.0}
{'loss': 2.2805, 'learning_rate': 8.857142857142857e-05, 'epoch': 2.0}


  0%|          | 0/58 [00:00<?, ?it/s]

...

{'eval_loss': 2.014413356781006, 'eval_runtime': 1.9895, 'eval_samples_per_second': 926.879, 'eval_steps_per_second': 29.153, 'epoch': 9.0}
{'loss': 2.1284, 'learning_rate': 2.3454157782515992e-07, 'epoch': 9.98}


  0%|          | 0/58 [00:00<?, ?it/s]

Overwriting existing adapter 'lora-mlm-target'.


{'eval_loss': 2.0134079456329346, 'eval_runtime': 1.9627, 'eval_samples_per_second': 939.529, 'eval_steps_per_second': 29.551, 'epoch': 10.0}
{'train_runtime': 345.6919, 'train_samples_per_second': 479.965, 'train_steps_per_second': 15.013, 'train_loss': 2.2125329842686883, 'epoch': 10.0}


TrainOutput(global_step=5190, training_loss=2.2125329842686883, metrics={'train_runtime': 345.6919, 'train_samples_per_second': 479.965, 'train_steps_per_second': 15.013, 'train_loss': 2.2125329842686883, 'epoch': 10.0})

In [22]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 7.41


In [23]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [25]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [26]:
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my appointment.
>>> i cancelled my tour.
>>> i cancelled my flight.
>>> i cancelled my meeting.
>>> i cancelled my engagement.
